In [84]:
#Importing Libraries
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
\
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

# 1. Reading Data

In [85]:
if os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))
else:
    print("please run the FB_EDA.ipynb or download the files from drive")

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


# 2 Graph features


## 2.1 Jaccard Distance (similarity measure)

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

Given any two sets, jaccard distance or jaccard similarity coefficient basically says:
 It is a statistic used for gauging the similarity of sample sets, which is the size of X intersection Y divided by size of X union Y.

In [4]:
# for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0 | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))) / (len(set(train_graph.successors(a)).union(set(train_graphsuccessors(b)))))
    except:
        return 0
    return sim

In [5]:
# one test case
print(jaccard_for_followees(273084,1505602))

0


In [6]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [9]:
print(jaccard_for_followers(273084,470294))

0


## 2.2 Cosine Distance(Otsuka Ochiai Coefficient)

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

- we will use a simple extension to cosine distance, which is Otsuka Ochiai Coefficient.
- Cosine Distance(Otsuka-Ochiai coefficient)= |𝑋∩𝑌|/ SQRT(|𝑋|⋅|𝑌|)
- Otsuka-Ochiai coefficient is used when X and Y are set.
- So, Cosine distance ( Otsuka-Ochiai Coefficient ) will be high when there is more overlap between sets X and Y.

In [10]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [11]:
print(cosine_for_followees(273084,1505602))

0.0


In [12]:
# For followers
def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [13]:
print(cosine_for_followers(669354,1635354))

0


## 2.3 PageRank

- PageRank works by counting number and quality of links to a page to determine a rough estimate of how important the page is.
- If a lot of pages are having destination as "B", then "B" must be important.
- If my page "B" is given as link in many important pages like C,E,D etc, then "B" page value increases.
- If a user has high pagerank score, then it implies that other users and highly important users are linking to Ui.
- PageRank can tell us about relative importance.

In [46]:
if not os.path.isfile(r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\page_rank.p"):
    pr = nx.pagerank(train_graph, alpha = 0.85)
    pickle.dump(pr,open(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\page_rank.p','wb'))
else:
    pr = pickle.load(open(r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\page_rank.p",'rb'))
    


In [47]:
print('min',pr[min(pr, key=pr.get)])
print('max',pr[max(pr, key=pr.get)])
print('mean',float(sum(pr.values())) / len(pr))

min 1.6556497245737814e-07
max 2.7098251341935827e-05
mean 5.615699699389075e-07


In [48]:
#for imputing to nodes which are not there in train data.
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)


5.615699699389075e-07


## 2.4 Shortest Path

- Shortest path measures how far two vertices are at the least.
- If there is no edge between two vertices , then we will consider default value as "-1".

In [49]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [50]:
#testing
compute_shortest_path_length(77697, 826021)

10

## 2.5 checking for same weakly connected component ( community )

In [51]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        for i in wcc:
            if b in i:
                index = i
                break
        if (b in index):
            train_graph.remove_edge(b,a)
            if compute_shortest_path_length(b,a)==-1:
                train_graph.add_edge(b,a)
                return 0
            else:
                train_graph.add_edge(b,a)
                return 1
        else:
            return 0
            
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

#### In the above code:
- First, we are checking if there is a direct edge from b to a. If yes, then we are checking do both b and a belong to same wcc or community.
- If yes, then we removed the direct edge from b to a and calculated the shortest path from b to a. If the shortest path does not exist then we are declaring that they are not in same wcc/community even though both of them belong to same wcc/community.
- However, if there exists a shortest path from b to a , then it means they could be friends and hence we are declaring them in wcc/community.
- secondly, we are checking that is there is a direct edge from a to b and follow the similar approach as above and return 1 or 0.
- If however, there is no direct edge from s to d or from d to s at the first place then we are simply checking that do they belong to same wcc. If yes, then just return 1 else 0.

In [52]:
belongs_to_same_wcc(861, 1659750)

0

In [27]:
belongs_to_same_wcc(669354,1635354)

0

## 2.6 Adamic/Adar Index

The Adamic/Adar Index is a measure designed to predict links in a social network.

In [36]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [37]:
calc_adar_in(1,189226)

0

In [38]:
calc_adar_in(669354,1635354)

0

## 2.7 Does the person follow back ?

If b is following a, then there is very high probability that a will follow b.
So if there is an edge between b and a, the function return 1 else 0. It's called the follows_back feature.

In [39]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [32]:
follows_back(1,189226)

1

In [33]:
follows_back(1,1635354)

0

## 2.8 Kartz Centrality 

Katz centrality computes the centrality for a node based on the centrality of it's neighbors. It's a generalization of eigenvector centrality.

In [40]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\katz.p'):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\katz.p','wb'))
else:
    katz = pickle.load(open(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\katz.p','rb'))

In [41]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007313532484065916
max 0.003394554981699122
mean 0.0007483800935562018


In [42]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007483800935562018


## 2.9 Hits Score

- HITS stands for Hyperlink-Induced Topic Search(often referred as hubs and authorities)is a link analysis algorithm that rates web pages.
- The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

In [43]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\hits.p'):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\hits.p','wb'))
else:
    hits = pickle.load(open(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\fea_sample\hits.p','rb'))

In [44]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


# 3 Feauturization

## 3.1 Reading a sample of data from both train and test

In [7]:
import os
import random
if os.path.isfile(r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\train_after_eda.csv"):
    filename = r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\train_after_eda.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 15100030
    n_train = sum(1 for line in open(filename)) #number of records in file (excludes header)
    s = 100000 #desired sample size
    skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039

In [8]:
if os.path.isfile(r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\test_after_eda.csv"):
    filename = r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\test_after_eda.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 3775008
    n_test = sum(1 for line in open(filename)) #number of records in file (excludes header)
    s = 50000 #desired sample size
    skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))
    #https://stackoverflow.com/a/22259008/4084039

In [9]:
print("Number of rows in the train data file:", n_train)
print("Number of rows we are going to elimiate in train data are",len(skip_train))
print("Number of rows in the test data file:", n_test)
print("Number of rows we are going to elimiate in test data are",len(skip_test))

Number of rows in the train data file: 15100030
Number of rows we are going to elimiate in train data are 15000030
Number of rows in the test data file: 3775008
Number of rows we are going to elimiate in test data are 3725008


In [13]:
df_final_train = pd.read_csv(r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\train_after_eda.csv", skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\train_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100001, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,65493,26839,1


In [15]:
df_final_test = pd.read_csv(r"C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\after_eda\test_after_eda.csv", skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\data\test_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_test.shape)
df_final_test.head(2)

Our train matrix size  (24964, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,539116,1687019,1


## 3.2 Adding Set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>jaccard_followers</li>
<li>jaccard_followees</li>
<li>cosine_followers</li>
<li>cosine_followees</li>
<li>num_followers_s</li>
<li>num_followees_s</li>
<li>num_followers_d</li>
<li>num_followees_d</li>
<li>inter_followers</li>
<li>inter_followees</li>
</ol>

In [27]:
def compute_features_stage1(df_final):
    #calculating no of followers followees for source and destination
    #calculating intersection of followers and followees for source and destination
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row['source_node']))
            s2=set(train_graph.successors(row['source_node']))
        except:
            s1 = set()
            s2 = set()
        try:
            d1=set(train_graph.predecessors(row['destination_node']))
            d2=set(train_graph.successors(row['destination_node']))
        except:
            d1 = set()
            d2 = set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))

        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))

        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    
    return  num_followers_s,num_followees_s,num_followers_d,num_followees_d,inter_followers,inter_followees

In [30]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage1.h5'):
    df_final_train['num_followers_s'], df_final_train['num_followers_d'], \
    df_final_train['num_followees_s'], df_final_train['num_followees_d'], \
    df_final_train['inter_followers'], df_final_train['inter_followees']= compute_features_stage1(df_final_train)
    
    df_final_test['num_followers_s'], df_final_test['num_followers_d'], \
    df_final_test['num_followees_s'], df_final_test['num_followees_d'], \
    df_final_test['inter_followers'], df_final_test['inter_followees']= compute_features_stage1(df_final_test)
    
    hdf = HDFStore(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage1.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()


In [23]:
pip install tables

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\sweta\anaconda3\envs\tf-gpu\python.exe -m pip install --upgrade pip' command.


In [31]:
df_final_train.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,inter_followers,inter_followees,num_followers_d
0,273084,1505602,1,0,0.000000,0.000000,0.000000,11,6,8,0,0,15
1,832016,1543415,1,0,0.187135,0.028382,0.343828,17,94,142,11,32,61
2,1325247,760242,1,0,0.369565,0.156957,0.566038,35,28,22,26,17,41
3,1368400,1006992,1,0,0.000000,0.000000,0.000000,2,11,7,0,0,5
4,140165,1708748,1,0,0.000000,0.000000,0.000000,5,1,3,0,0,11


In [32]:
a=df_final_train['num_followers_s'].values
b=df_final_train['num_followers_d'].values
for x,y in (zip(a,b)):
  if x==0:
    if y!=0:
      print('i')


i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i


In [33]:
np.count_nonzero(a)

89571

In [34]:
np.count_nonzero(b)

89361

## 3.3 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>adar index</li>
<li>is following back</li>
<li>belongs to same weakly connect components</li>
<li>shortest path between source and destination</li>
</ol>

In [54]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage2.h5'):
    #mapping adar index on train
    df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)
    #mapping adar index on test
    df_final_test['adar_index'] = df_final_test.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping followback or not on train
    df_final_train['follows_back'] = df_final_train.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)

    #mapping followback or not on test
    df_final_test['follows_back'] = df_final_test.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping same component of wcc or not on train
    df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)

    ##mapping same component of wcc or not on train
    df_final_test['same_comp'] = df_final_test.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
    
    #--------------------------------------------------------------------------------------------------------
    #mapping shortest path on train 
    df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)
    #mapping shortest path on test
    df_final_test['shortest_path'] = df_final_test.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)

    hdf = HDFStore(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage2.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()

In [55]:
df_final_train.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,inter_followers,inter_followees,num_followers_d,adar_index,follows_back,same_comp,shortest_path
0,273084,1505602,1,0,0.000000,0.000000,0.000000,11,6,8,0,0,15,0.000000,0,1,4
1,832016,1543415,1,0,0.187135,0.028382,0.343828,17,94,142,11,32,61,16.362912,0,1,2
2,1325247,760242,1,0,0.369565,0.156957,0.566038,35,28,22,26,17,41,10.991826,0,1,2
3,1368400,1006992,1,0,0.000000,0.000000,0.000000,2,11,7,0,0,5,0.000000,0,1,6
4,140165,1708748,1,0,0.000000,0.000000,0.000000,5,1,3,0,0,11,0.000000,0,1,-1


## 3.4 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>Weight Features
    <ul>
        <li>weight of incoming edges</li>
        <li>weight of outgoing edges</li>
        <li>weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges * weight of outgoing edges</li>
        <li>2*weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges + 2*weight of outgoing edges</li>
    </ul>
</li>
<li>Page Ranking of source</li>
<li>Page Ranking of dest</li>
<li>katz of source</li>
<li>katz of dest</li>
<li>hubs of source</li>
<li>hubs of dest</li>
<li>authorities_s of source</li>
<li>authorities_s of dest</li>
</ol>

In [56]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|████████████████████████████████████████████████████████████████████| 1780722/1780722 [00:12<00:00, 137419.73it/s]


In [57]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage3.h5'):
    #mapping to pandas train
    df_final_train['weight_in'] = df_final_train.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_final_train['weight_out'] = df_final_train.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

    #mapping to pandas test
    df_final_test['weight_in'] = df_final_test.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_final_test['weight_out'] = df_final_test.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))


    #some features engineerings on the in and out weights
    df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
    df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
    df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
    df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)

    #some features engineerings on the in and out weights
    df_final_test['weight_f1'] = df_final_test.weight_in + df_final_test.weight_out
    df_final_test['weight_f2'] = df_final_test.weight_in * df_final_test.weight_out
    df_final_test['weight_f3'] = (2*df_final_test.weight_in + 1*df_final_test.weight_out)
    df_final_test['weight_f4'] = (1*df_final_test.weight_in + 2*df_final_test.weight_out)

In [58]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage3.h5'):
    
    #page rank for source and destination in Train and Test
    #if anything not there in train graph then adding mean page rank 
    df_final_train['page_rank_s'] = df_final_train.source_node.apply(lambda x:pr.get(x,mean_pr))
    df_final_train['page_rank_d'] = df_final_train.destination_node.apply(lambda x:pr.get(x,mean_pr))

    df_final_test['page_rank_s'] = df_final_test.source_node.apply(lambda x:pr.get(x,mean_pr))
    df_final_test['page_rank_d'] = df_final_test.destination_node.apply(lambda x:pr.get(x,mean_pr))
    #================================================================================

    #Katz centrality score for source and destination in Train and test
    #if anything not there in train graph then adding mean katz score
    df_final_train['katz_s'] = df_final_train.source_node.apply(lambda x: katz.get(x,mean_katz))
    df_final_train['katz_d'] = df_final_train.destination_node.apply(lambda x: katz.get(x,mean_katz))

    df_final_test['katz_s'] = df_final_test.source_node.apply(lambda x: katz.get(x,mean_katz))
    df_final_test['katz_d'] = df_final_test.destination_node.apply(lambda x: katz.get(x,mean_katz))
    #================================================================================

    #Hits algorithm score for source and destination in Train and test
    #if anything not there in train graph then adding 0
    df_final_train['hubs_s'] = df_final_train.source_node.apply(lambda x: hits[0].get(x,0))
    df_final_train['hubs_d'] = df_final_train.destination_node.apply(lambda x: hits[0].get(x,0))

    df_final_test['hubs_s'] = df_final_test.source_node.apply(lambda x: hits[0].get(x,0))
    df_final_test['hubs_d'] = df_final_test.destination_node.apply(lambda x: hits[0].get(x,0))
    #================================================================================

    #Hits algorithm score for source and destination in Train and Test
    #if anything not there in train graph then adding 0
    df_final_train['authorities_s'] = df_final_train.source_node.apply(lambda x: hits[1].get(x,0))
    df_final_train['authorities_d'] = df_final_train.destination_node.apply(lambda x: hits[1].get(x,0))

    df_final_test['authorities_s'] = df_final_test.source_node.apply(lambda x: hits[1].get(x,0))
    df_final_test['authorities_d'] = df_final_test.destination_node.apply(lambda x: hits[1].get(x,0))
    #================================================================================

    hdf = HDFStore(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage3.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()

In [60]:
df_final_train.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,...,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d
0,273084,1505602,1,0,0.000000,0.000000,0.000000,11,6,8,...,1.005929,0.877964,2.045290e-06,3.459963e-07,0.000773,0.000756,1.943132e-13,1.941103e-13,9.226339e-16,2.231877e-15
1,832016,1543415,1,0,0.187135,0.028382,0.343828,17,94,142,...,0.332196,0.356598,2.353458e-07,6.427660e-07,0.000845,0.001317,3.906648e-11,9.424102e-11,1.208074e-11,1.273080e-10
2,1325247,760242,1,0,0.369565,0.156957,0.566038,35,28,22,...,0.525694,0.494302,6.211019e-07,5.179801e-07,0.000885,0.000855,7.730764e-114,4.067322e-114,2.681298e-113,2.199205e-113
3,1368400,1006992,1,0,0.000000,0.000000,0.000000,2,11,7,...,0.985599,1.105172,2.998153e-07,1.704245e-06,0.000739,0.000773,5.443738e-17,4.139999e-16,2.413250e-14,6.688064e-15
4,140165,1708748,1,0,0.000000,0.000000,0.000000,5,1,3,...,2.301511,1.603023,4.349180e-07,2.089590e-07,0.000751,0.000735,3.887821e-16,4.721269e-16,7.552255e-16,2.734009e-18


## 3.5 Adding new set of features

__we will create these each of these features for both train and test data points__
##### SVD FEATURES FOR SOURCE NODES AND DESTINATION NODES

- In our case, our adjacency matric is binary matrix.
- There are other variation of matrix for undirected graphs called weighted graphs, for now let's stick to directed graphs.
- We will get very large and sparse matrix as most of the values will be zero.
- We will decompose the adjacency matrix using SVD.
- A = U Σ VT
- Where U = size of 1.78M x 6
- Σ = size of 6 x 6
- VT(transpose) = size of 6 x 1.78M
- In A, U's ith row represents the same size vector as VT's ith column. Both are the same size vector representation of ith vertex of the graph. We got this using left singular matrix U and right singular matrix VT.
- So, for the pair of vertices where we need to predict if there is an edge or not, we have 24 features (for Ui-u-6,vt-6, Uj-u-6,vt-6).

In [61]:
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

In [62]:
#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [63]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [64]:
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780722, 1780722)
U Shape (1780722, 6)
V Shape (6, 1780722)
s Shape (6,)


In [65]:
if not os.path.isfile(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage4.h5'):
    #===================================================================================================
    
    df_final_train[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_train.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
    df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    #===================================================================================================
    
    df_final_train[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
    df_final_train.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

    df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================
    
    df_final_test[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_test.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
    df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)

    #===================================================================================================
    
    df_final_test[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
    df_final_test.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

    df_final_test[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================

    hdf = HDFStore(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\storage_sample_stage4.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()

In [78]:
df_final_train.to_csv(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\df_final_train.csv')

In [87]:
df_final_test.to_csv(r'C:\Users\sweta\OneDrive\Desktop\ML projects\Facebook Friend Recommendation using Graph Mining\mine\df_final_test.csv')

In [76]:
pd.set_option("display.max_columns", None)

#### Final Data

In [86]:
df_final_test.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,inter_followers,inter_followees,num_followers_d,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,svd_u_s_1,svd_u_s_2,svd_u_s_3,svd_u_s_4,svd_u_s_5,svd_u_s_6,svd_u_d_1,svd_u_d_2,svd_u_d_3,svd_u_d_4,svd_u_d_5,svd_u_d_6,svd_v_s_1,svd_v_s_2,svd_v_s_3,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6
0,848424,784690,1,0,0.0,0.029161,0.000000,6,14,9,1,0,6,0.000000,1,1,2,0.258199,0.377964,0.636163,0.097590,0.894362,1.014128,6.557971e-07,0.000002,0.000754,0.000786,3.243237e-16,1.745627e-16,2.969838e-15,9.269213e-14,-9.987979e-12,2.283678e-13,1.439968e-10,6.136162e-13,4.188172e-13,5.983503e-15,-1.026186e-11,5.316373e-13,5.843225e-10,1.166047e-13,2.253356e-11,3.220476e-15,-2.148852e-13,1.883148e-13,5.904807e-11,2.701538e-12,4.341620e-13,5.535503e-14,-9.994076e-10,5.791910e-10,3.512364e-07,2.486658e-09,2.771146e-09,1.727694e-12
1,483294,1255532,1,0,0.0,0.000000,0.000000,2,17,19,0,0,1,0.000000,1,1,7,0.235702,0.707107,0.942809,0.166667,1.178511,1.649916,2.172064e-07,0.000001,0.000739,0.000801,1.702625e-19,2.706300e-15,2.522357e-16,5.277458e-15,-2.079153e-15,7.009827e-16,6.970710e-10,9.561783e-15,1.728827e-15,3.104892e-18,-1.960277e-11,2.861530e-11,1.939773e-08,1.907382e-12,3.797410e-11,4.992899e-14,-4.054309e-13,2.895682e-13,2.545382e-10,2.248568e-14,3.600957e-13,4.701438e-15,-9.360516e-12,3.206809e-10,4.668696e-08,6.665777e-12,1.495979e-10,9.836673e-14
2,626190,1729265,1,0,0.0,0.000000,0.000000,15,10,9,0,0,16,0.000000,0,1,5,0.301511,0.242536,0.544047,0.073127,0.845558,0.786583,1.853369e-06,0.000002,0.000789,0.000770,9.426796e-14,4.116616e-16,2.253244e-15,2.079387e-15,-1.782290e-11,2.389627e-11,1.443050e-06,2.358030e-12,2.689638e-12,1.739178e-12,-1.863933e-13,1.806785e-13,2.544597e-07,1.693296e-13,2.712750e-13,7.594708e-15,-4.148102e-13,4.618813e-12,1.122150e-05,1.778927e-12,2.740535e-13,4.199835e-14,-4.253076e-13,4.789463e-13,3.479824e-07,1.630550e-13,3.954708e-13,3.875785e-14
3,947219,425228,1,0,0.0,0.000000,0.000000,11,37,34,0,0,10,0.000000,0,1,3,0.162221,0.301511,0.463733,0.048912,0.625954,0.765244,7.000791e-07,0.000002,0.000778,0.000884,9.876114e-14,1.039593e-13,1.511694e-14,3.478438e-14,-4.491246e-11,9.917408e-13,7.891250e-06,9.458625e-12,2.715848e-11,1.822074e-12,-1.070733e-10,2.888035e-12,2.070703e-05,7.229355e-12,3.563659e-11,1.917977e-12,-8.942488e-12,5.535630e-12,5.223676e-06,7.917168e-13,4.020707e-12,2.817657e-13,-2.162590e-11,6.939194e-12,1.879861e-05,4.384817e-12,1.239414e-11,6.483486e-13
4,991374,975044,1,0,0.2,0.042767,0.347833,12,27,27,4,7,15,6.136433,0,1,2,0.188982,0.250000,0.438982,0.047246,0.627964,0.688982,7.103008e-07,0.000001,0.000779,0.000840,1.557332e-15,1.096037e-14,5.180869e-15,1.296135e-14,-9.778445e-13,5.724609e-13,4.951883e-06,1.396835e-12,2.153249e-11,2.873164e-14,-2.275780e-11,1.007615e-12,2.517628e-06,1.394103e-10,2.961969e-11,2.022111e-13,-3.804979e-12,1.593011e-13,1.035016e-06,1.361574e-13,1.154623e-12,9.656659e-14,-8.742904e-12,7.467370e-12,1.256880e-05,3.636983e-12,3.948463e-12,2.415862e-13


In [82]:
df_final_train.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,inter_followers,inter_followees,num_followers_d,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,svd_u_s_1,svd_u_s_2,svd_u_s_3,svd_u_s_4,svd_u_s_5,svd_u_s_6,svd_u_d_1,svd_u_d_2,svd_u_d_3,svd_u_d_4,svd_u_d_5,svd_u_d_6,svd_v_s_1,svd_v_s_2,svd_v_s_3,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6
0,273084,1505602,1,0,0.000000,0.000000,0.000000,11,6,8,0,0,15,0.000000,0,1,4,0.377964,0.250000,0.627964,0.094491,1.005929,0.877964,2.045290e-06,3.459963e-07,0.000773,0.000756,1.943132e-13,1.941103e-13,9.226339e-16,2.231877e-15,-1.666225e-13,4.613398e-13,1.043044e-05,6.676964e-13,2.451081e-13,3.584580e-12,-2.038017e-11,5.397496e-13,1.068946e-06,1.192357e-12,2.279486e-12,3.580838e-12,-7.216133e-13,3.925975e-13,1.983691e-06,1.545076e-13,8.108434e-13,1.719702e-14,-1.355368e-12,4.675307e-13,1.128591e-06,6.616555e-14,9.771077e-13,4.159752e-14
1,832016,1543415,1,0,0.187135,0.028382,0.343828,17,94,142,11,32,61,16.362912,0,1,2,0.102598,0.127000,0.229598,0.013030,0.332196,0.356598,2.353458e-07,6.427660e-07,0.000845,0.001317,3.906648e-11,9.424102e-11,1.208074e-11,1.273080e-10,7.050100e-13,-8.250521e-11,-1.717718e-10,3.705016e-02,1.032391e-11,7.207496e-10,1.644185e-12,-2.047743e-10,-3.943972e-10,8.617541e-02,2.399539e-11,1.738682e-09,2.685511e-13,-3.316833e-11,-6.236106e-11,1.345726e-02,3.703477e-12,2.251737e-10,1.244920e-12,-1.636940e-10,-3.112679e-10,6.738902e-02,2.607800e-11,2.372904e-09
2,1325247,760242,1,0,0.369565,0.156957,0.566038,35,28,22,26,17,41,10.991826,0,1,2,0.185695,0.154303,0.339999,0.028653,0.525694,0.494302,6.211019e-07,5.179801e-07,0.000885,0.000855,7.730764e-114,4.067322e-114,2.681298e-113,2.199205e-113,9.948737e-20,6.191958e-19,3.317490e-18,-1.439229e-18,2.875334e-18,-4.021771e-18,-3.697725e-20,-3.239749e-20,1.583095e-18,-5.367094e-19,2.082609e-18,-1.855474e-18,9.173708e-20,7.848290e-20,1.160581e-18,-3.471777e-19,9.944941e-19,-8.542327e-19,7.342034e-20,7.065210e-20,9.502831e-19,-2.884729e-19,7.955298e-19,-6.964203e-19
3,1368400,1006992,1,0,0.000000,0.000000,0.000000,2,11,7,0,0,5,0.000000,0,1,6,0.288675,0.408248,0.696923,0.117851,0.985599,1.105172,2.998153e-07,1.704245e-06,0.000739,0.000773,5.443738e-17,4.139999e-16,2.413250e-14,6.688064e-15,-9.965436e-12,4.077137e-13,5.083778e-10,1.985269e-14,2.471968e-12,1.004307e-15,-2.629030e-13,1.428940e-11,9.146784e-11,8.590132e-12,2.406658e-10,7.638097e-15,-2.170899e-13,5.206150e-13,6.058498e-11,1.514614e-11,1.513483e-12,4.498061e-13,-9.818087e-10,3.454672e-11,5.213635e-08,9.595823e-13,3.047045e-10,1.246592e-13
4,140165,1708748,1,0,0.000000,0.000000,0.000000,5,1,3,0,0,11,0.000000,0,1,-1,1.000000,0.301511,1.301511,0.301511,2.301511,1.603023,4.349180e-07,2.089590e-07,0.000751,0.000735,3.887821e-16,4.721269e-16,7.552255e-16,2.734009e-18,-3.408942e-13,1.125209e-13,3.190883e-07,8.550847e-14,1.405352e-13,7.172422e-15,-2.550169e-13,2.424636e-13,2.436301e-07,3.679956e-14,1.348518e-13,8.710475e-15,-4.795514e-13,1.521297e-13,1.197283e-07,1.999809e-14,3.360247e-13,1.407670e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
